# Это проект по поиску друзей по интересам в группах Вконтакте. 
Загрузим необходимые библиотеки:

In [24]:
import vk
import pandas as pd

Подключим VK API к проекту:

In [25]:
VK_TOKEN = '3666e68f93e125d24e29ffd94b50bfbc79f3f0ee64b5a2076fbc5d92cd1d2b20d5a089b25b205512897d6'
ID = 10385605
session = vk.Session(VK_TOKEN)
vk_api = vk.API(session, v=5.103)

Определим методы для дальнейшей работы:

Получаем типы групп по айди пользователя.

In [26]:
def get_groups_activity(id):
    groups = vk_api.groups.get(user_id=id, fields=['activity'], extended=1)
    activities = []
    for group in groups['items']:
        try:
            activities.append(group['activity'])
        except Exception:
            continue
    return activities

Получаем процентное соотношение конкретного типа группы ко всем группам. Выводим первые 10 человек

In [27]:
def vadim_gets_percents(activities):
    df = pd.DataFrame(data= activities, columns=['activity'])
    df = df.loc[(df.loc[:, 'activity'] != 'Открытая группа') & (df.loc[:, 'activity'] != 'Закрытая группа')]
    df['count'] = 1
    df2 = df.groupby('activity').count()
    df2['percent'] = df2['count'] / len(df.index) * 100
    df2 = df2.sort_values(by = 'percent', ascending = False)
    activities = df2.index
    df2['activity'] = activities
    df2['index'] = range(len(df2))
    df2 = df2.set_index('index')
    return df2.head(10)

Вычисляем сколько групп в процентном соотношении соотвествуют между мной и другом

In [28]:
def get_my_libe_froind(first_p, second_p):
    count = 0
    for i in range(len(first_p)):  
        for j in range(len(second_p)):
            if (first_p.loc[i, 'activity'] == second_p.loc[j, 'activity']) &  (abs(first_p.loc[i, 'percent'] - second_p.loc[j, 'percent']) < 5):
                count += 1
    return count

Получаем таблицу с процентным соотношением типов групп для себя.

In [29]:
my_activities = get_groups_activity(ID)
my_percents = vadim_gets_percents(my_activities)
my_percents

,count,percent,activity
index,,,
0,12,10.344828,Образование
1,11,9.482759,Интернет-СМИ
2,9,7.758621,Музыкант
3,6,5.172414,Творчество
4,5,4.310345,Музыкальная группа
5,4,3.448276,Музыка
6,4,3.448276,Юмор
7,4,3.448276,Наука
8,3,2.586207,Искусство и развлечения


Получаем имена друзей из апи

In [30]:
friends = vk_api.friends.get(user_id=ID, extended=1, fields='nickname')['items']
df = pd.DataFrame(columns=['friend_name', 'count'])
i = 0

Проходим по каждому другу. И вычисляем сколько у каждого друга общих типов групп с нашими. Выводим топ друзей.

In [31]:
for friend in friends: 
    try:
        fr_activities = get_groups_activity(friend['id'])
        fr_percents = vadim_gets_percents(fr_activities)
        friend_name = friend['first_name'] + ' ' + friend['last_name']
        count = get_my_libe_froind(my_percents, fr_percents)
        df.loc[i] = [friend_name, count]
        i += 1
    except Exception:
        continue

df = df.sort_values(by='count', ascending=False)
print(df.head(50))

                          friend_name count
90                  Тимур Хайрутдинов     6
97                       Аня Изгарова     6
5    Айдар Гайнутдинов-Rammsteinovich     5
96                   Георгий Игоревич     5
80                      Эмиль Рахимов     4
89                 Анастасия Самилова     4
64                       Ксюша Панова     4
93                    Катя Покровская     4
95                     Булат Нугманов     4
23                       Азат Булатов     4
82                      Тимур Гвоздев     4
29                      Ильяс Мустаев     4
2                    Михаил Абрамский     4
65                        Егор Финеев     4
103                 Родион Кантарьянц     4
76                       Эмиль Кулиев     3
46                    Богдан Сошников     3
60                  Вадим Иванаевский     3
59                    Настя Журавлева     3
33                     Михаил Золотов     3
66                        Артём Лялин     3
86                  Гульшат Шамг

In [32]:
df

,friend_name,count
90,Тимур Хайрутдинов,6
97,Аня Изгарова,6
5,Айдар Гайнутдинов-Rammsteinovich,5
96,Георгий Игоревич,5
80,Эмиль Рахимов,4
...,...,...
36,Руслан Нигматзянов,0
75,Богдан Полин,0
43,Эльмира Баширова,0
79,Илья Тарасенко,0


In [33]:
df = df.drop_duplicates(subset=['friend_name'])
df

,friend_name,count
90,Тимур Хайрутдинов,6
97,Аня Изгарова,6
5,Айдар Гайнутдинов-Rammsteinovich,5
96,Георгий Игоревич,5
80,Эмиль Рахимов,4
...,...,...
36,Руслан Нигматзянов,0
75,Богдан Полин,0
43,Эльмира Баширова,0
79,Илья Тарасенко,0
